In [1]:
from torchHMM.model.discretized_HMM import DiscreteHMM, DISCRETIZATION_TECHNIQUES
from torchHMM.model.discretized_flow_HMM import FlowHMM

In [2]:
import numpy as np
import pandas as pd
import pickle as pkl

In [3]:
from hmmlearn.hmm import CategoricalHMM, GaussianHMM
from gensim.models import KeyedVectors


In [4]:
with open('clickstream_experiment/data/preprocessed_data/ClickStream_test_cleaned.pkl', 'rb') as f:
    cs = pkl.load(f)

In [5]:
session_train, sessions_test = cs.sessions[:10000], cs.sessions[10000:11000]


In [6]:
item_ids = np.unique([e.item_id for s in session_train for e in s.event_list])
item_ids = {item_ids[i]: i for i in range(len(item_ids))}

Xs = [[item_ids[e.item_id] for e in s.event_list] for s in session_train]
l = [len(y) for y in Xs]

X = np.concatenate(Xs).reshape(-1, 1)

target =  [[item_ids[e.item_id] for e in s.event_list if e.item_id in item_ids.keys()] for s in sessions_test]
target = [l for l in target if len(l)]
lt = np.array([len(y) for y in target])
target = np.concatenate(target).reshape(-1, 1)

In [7]:
len(item_ids.keys())

40682

In [8]:
chmm = CategoricalHMM(5, n_iter=10, verbose=True)


In [9]:
%time chmm.fit(X, l)

Fitting a model with 203429 free scalar parameters with only 101294 data points will result in a degenerate solution.
         1    -1086149.4128             +nan
         2    -1017010.9627      +69138.4501
         3    -1006078.0874      +10932.8753
         4     -997579.3790       +8498.7084
         5     -991854.2349       +5725.1441
         6     -988110.1798       +3744.0551
         7     -985622.8717       +2487.3081
         8     -983949.0090       +1673.8628
         9     -982787.7529       +1161.2561


CPU times: user 14.4 s, sys: 3.66 ms, total: 14.4 s
Wall time: 14.5 s


        10     -981980.0398        +807.7131


CategoricalHMM(n_components=5,
               random_state=RandomState(MT19937) at 0x7FE9D7CB4A40,
               verbose=True)

In [10]:
print(chmm.score(target, lt))

-60913.77072549441


In [11]:
vectors = KeyedVectors.load(
    f"clickstream_experiment/data/preprocessed_data/vectors_train_20_10_5_8_cleaned.kv"
)
vecs = np.concatenate(
    [
        vectors.get_vector(k).reshape(1, -1)
        for k in list(vectors.key_to_index.keys())
    ]
)

In [12]:
unknown = vecs.mean(axis=0).reshape(1, -1)

def get_vec(e):
    try:
        return vectors.get_vector(e.item_id).reshape(1, -1)
    except:
        return unknown

In [13]:
Xs_c = [[get_vec(e) for e in s.event_list] for s in session_train]
l_c = np.array([len(y) for y in X])

X_c = np.concatenate([a for as_ in Xs_c for a in as_], axis=0)

target_c =  [[get_vec(e) for e in s.event_list if e.item_id in item_ids.keys()] for s in sessions_test]
target_c = np.concatenate([a for as_ in target_c for a in as_], axis=0)


In [14]:
w2vhmm = GaussianHMM(5, n_iter=10, verbose=True)

In [15]:
%time w2vhmm.fit(X_c, l_c)

/ziob/klaudia/miniconda3/envs/preproc/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
         1     2380636.8370             +nan
         2     4816711.6964    +2436074.8594
         3     8651356.4184    +3834644.7220
         4    13754777.6574    +5103421.2390
         5    13755287.9813        +510.3239
         6    13755967.3509        +679.3696
         7    13756602.2661        +634.9152
         8    13757452.8608        +850.5947
         9    13758106.8537        +653.9929
        10    13759075.1177        +968.2640
Some rows of transmat_ have zero sum because no transition from the state was ever observed.


CPU times: user 2min 5s, sys: 741 ms, total: 2min 6s
Wall time: 2min 6s


GaussianHMM(n_components=5, verbose=True)

In [16]:
w2vhmm.transmat_

array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])

In [17]:
zeros = w2vhmm.transmat_.sum(axis=1) == 0
w2vhmm.transmat_[zeros] = np.ones(w2vhmm.transmat_.shape)[zeros] / w2vhmm.n_components


In [18]:
print(w2vhmm.score(target_c, lt))

625033.6468025462


In [19]:
w2v_dhmm = DiscreteHMM(n_components=5, no_nodes=64, optimizer="Adam", verbose=True, covariance_type='diag')

In [20]:
%time w2v_dhmm.fit(X_c, l_c)

/ziob/klaudia/miniconda3/envs/preproc/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/ziob/klaudia/FlowHMM/torchHMM/model/discretized_HMM.py:447: RuntimeWarning: invalid value encountered in divide
  cooc_matrix /= cooc_matrix.sum()


ValueError: 'diag' covars must have shape (n_components, n_dim)

In [21]:
print(w2v_dhmm.score(target_c, lt))

nan


In [22]:
w2v_fhmm = FlowHMM(n_components=5, no_nodes=512, l=100, optimizer="Adam", verbose=True)

%time w2v_fhmm.fit(X_c, l_c)

print(w2v_fhmm.score(target_c, lt))


/ziob/klaudia/FlowHMM/torchHMM/model/discretized_flow_HMM.py:417: RuntimeWarning: invalid value encountered in divide
  cooc_matrix /= cooc_matrix.sum()


RuntimeError: Index put requires the source and destination dtypes match, got Double for the destination and Float for the source.

TypeError: FlowHMM.discretize() missing 1 required positional argument: 'force'